In [42]:
#importing libraries
import numpy as numpy
import pandas as pandas

data = pandas.read_csv('smsdata.tsv', sep='\t')
data.head()

,lab,msg,len,pun
0,ham,"Go until jurong point, crazy.. Available only ...",111,9
1,ham,Ok lar... Joking wif u oni...,29,6
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155,6
3,ham,U dun say so early hor... U c already then say...,49,6
4,ham,"Nah I don't think he goes to usf, he lives aro...",61,2


In [43]:
data.isnull().sum()

lab    0
msg    0
len    0
pun    0
dtype: int64

In [44]:
data['lab'].value_counts()

ham     4825
spam     747
Name: lab, dtype: int64

In [45]:
from sklearn.model_selection import train_test_split

X = data['msg']  
y = data['lab']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [46]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()

X_train_counts = count_vect.fit_transform(X_train)
X_train_counts.shape

(3733, 7082)

In [47]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()

X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(3733, 7082)

In [48]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

X_train_tfidf = vectorizer.fit_transform(X_train) 
X_train_tfidf.shape

(3733, 7082)

In [49]:
from sklearn.svm import LinearSVC
clf = LinearSVC()
clf.fit(X_train_tfidf,y_train)

LinearSVC()

In [50]:
from sklearn.pipeline import Pipeline


text_clf = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', LinearSVC()),
])

text_clf.fit(X_train, y_train)  

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', LinearSVC())])

In [51]:
predictions = text_clf.predict(X_test)

In [52]:
#displaying data in a metrics
from sklearn import metrics
print(metrics.confusion_matrix(y_test,predictions))

[[1586    7]
 [  12  234]]


In [53]:
#Accuracy in a metrics
print(metrics.classification_report(y_test,predictions))

              precision    recall  f1-score   support

         ham       0.99      1.00      0.99      1593
        spam       0.97      0.95      0.96       246

    accuracy                           0.99      1839
   macro avg       0.98      0.97      0.98      1839
weighted avg       0.99      0.99      0.99      1839



In [54]:
# Print the overall accuracy
print(metrics.accuracy_score(y_test,predictions))

0.989668297988037


In [55]:
#sample data of ham
smsham = """Wait that's still not all that clear,
            were you not sure about me being sarcastic or
            that that's why x doesn't want to live with us"""

In [56]:
print(text_clf.predict([smsham]))

['ham']


In [57]:
#sample data of span
smsspam = """Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005.
             Text FA to 87121 to receive entry question(std txt rate)T&C's apply
             08452810075over18's"""

In [58]:
print(text_clf.predict([smsspam]))

['spam']


In [59]:
#sample data of ham
smsham1 = "Going for dinner.msg you after."

In [60]:
print(text_clf.predict([smsham1]))

['ham']


In [61]:
#sample data of spam
smsham2 = "As a valued customer, I am pleased to advise you that following recent review of your Mob No. you are awarded with a £1500 Bonus Prize, call 09066364589"

In [62]:
print(text_clf.predict([smsham2]))

['spam']
